In [1]:

from scipy.io import arff
import sys 
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt #used to plot graphs
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.cross_validation import KFold # use for cross validation
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import LSTM

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#load the dataset
data = arff.loadarff('C:\Anaconda3\EEG_Eye_State.arff')
df = pd.DataFrame(data[0])

# Pre processing

In [3]:
columns = []
for i in columns:
    df[i] = df[i].astype('category')
for i in columns:
    df[i] = pd.Categorical(df[i]).codes

In [4]:
#display first 5 data
df.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,b'0'
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,b'0'
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,b'0'
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,b'0'
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,b'0'


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14980 entries, 0 to 14979
Data columns (total 15 columns):
AF3             14980 non-null float64
F7              14980 non-null float64
F3              14980 non-null float64
FC5             14980 non-null float64
T7              14980 non-null float64
P7              14980 non-null float64
O1              14980 non-null float64
O2              14980 non-null float64
P8              14980 non-null float64
T8              14980 non-null float64
FC6             14980 non-null float64
F4              14980 non-null float64
F8              14980 non-null float64
AF4             14980 non-null float64
eyeDetection    14980 non-null object
dtypes: float64(14), object(1)
memory usage: 1.7+ MB


In [6]:
df.dtypes

AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6             float64
F4              float64
F8              float64
AF4             float64
eyeDetection     object
dtype: object

In [7]:
df.shape

(14980, 15)

In [8]:
df.columns

Index(['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6',
       'F4', 'F8', 'AF4', 'eyeDetection'],
      dtype='object')

In [9]:
#check if there are coloumns with missing values
droping_list_all=[]
for j in range(0,14):
    if not df.iloc[:, j].notnull().all():
        droping_list_all.append(j)        
        #print(df.iloc[:,j].unique())
droping_list_all

[]

# Data Preparation for LSTM Model 1

In [10]:
X = np.zeros((14, len(df)))
for i in range(len(df.columns)-1):
    X[i] = np.array(df[df.columns[i]])
X = X.transpose()

In [11]:
dat1 = [[X[i+j] for i in range (10)]for j in range(len(df)-10)]
X =np.array(dat1)
X

array([[[4329.23, 4009.23, 4289.23, ..., 4280.51, 4635.9 , 4393.85],
        [4324.62, 4004.62, 4293.85, ..., 4279.49, 4632.82, 4384.1 ],
        [4327.69, 4006.67, 4295.38, ..., 4282.05, 4628.72, 4389.23],
        ...,
        [4325.64, 4006.67, 4278.46, ..., 4266.67, 4622.05, 4380.51],
        [4326.15, 4010.77, 4276.41, ..., 4273.85, 4627.18, 4389.74],
        [4326.15, 4011.28, 4276.92, ..., 4277.95, 4637.44, 4393.33]],

       [[4324.62, 4004.62, 4293.85, ..., 4279.49, 4632.82, 4384.1 ],
        [4327.69, 4006.67, 4295.38, ..., 4282.05, 4628.72, 4389.23],
        [4328.72, 4011.79, 4296.41, ..., 4287.69, 4632.31, 4396.41],
        ...,
        [4326.15, 4010.77, 4276.41, ..., 4273.85, 4627.18, 4389.74],
        [4326.15, 4011.28, 4276.92, ..., 4277.95, 4637.44, 4393.33],
        [4326.15, 4010.77, 4272.82, ..., 4272.82, 4631.79, 4382.56]],

       [[4327.69, 4006.67, 4295.38, ..., 4282.05, 4628.72, 4389.23],
        [4328.72, 4011.79, 4296.41, ..., 4287.69, 4632.31, 4396.41],
    

In [12]:
Y = np.array(df[df.columns[14]],dtype=np.int8)

In [13]:
target = [[Y[i+j] for i in range (1)]for j in range(len(df)-10)]
Y = np.array(target, dtype =int)

In [14]:
Y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

# Train Test Split

In [15]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [16]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(11976, 10, 14)
(2994, 10, 14)
(11976, 10)
(2994, 10)


# Define the LSTM Model 1

In [17]:
model1 = Sequential()
model1.add(LSTM((1),batch_input_shape = (None,10,14),return_sequences = False,activation='relu'))
model1.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

In [18]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1)                 64        
Total params: 64
Trainable params: 64
Non-trainable params: 0
_________________________________________________________________


# Train LSTM Model 1

In [19]:
model1.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Train on 11976 samples, validate on 2994 samples
Epoch 1/50
11976/11976 [==============================] - 2s 166us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - val_acc: 0.5538
Epoch 2/50
11976/11976 [==============================] - 2s 127us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - val_acc: 0.5538
Epoch 3/50
11976/11976 [==============================] - 2s 131us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - val_acc: 0.5538
Epoch 4/50
11976/11976 [==============================] - 2s 131us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - val_acc: 0.5538
Epoch 5/50
11976/11976 [==============================] - 1s 121us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - val_acc: 0.5538
Epoch 6/50
11976/11976 [==============================] - 1s 121us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - val_acc: 0.5538
Epoch 7/50
11976/11976 [==============================] - 2s 148us/step - loss: 0.4490 - acc: 0.5510 - val_loss: 0.4462 - v

# Predict using LSTM Model 1

In [20]:
predict1 = model1.predict(x_test)

In [21]:
predict1

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [22]:
eval1 = model1.evaluate(x_test, y_test, verbose=1)

2994/2994 [==============================] - 0s 31us/step


In [32]:
print('Test score:', eval1[0])
print('Test accuracy:', eval1[1])

Test score: 0.44622578490313963
Test accuracy: 0.5537742150968604


# Data Preparation for LSTM Model 2


In [13]:
target = [[Y[i+j] for i in range (10)]for j in range(len(df)-10)]
Y = np.array(target, dtype =int)

In [14]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

# Define the LSTM Model 2

In [17]:
model2 = Sequential()
model2.add(LSTM((10),batch_input_shape = (None,10,14),return_sequences = False))
#model2.add(LSTM((1),return_sequences = False))
model2.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

In [18]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                1000      
Total params: 1,000
Trainable params: 1,000
Non-trainable params: 0
_________________________________________________________________


# Train LSTM Model 2

In [19]:
model2.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Train on 11976 samples, validate on 2994 samples
Epoch 1/50
11976/11976 [==============================] - 2s 173us/step - loss: 0.4369 - acc: 9.1850e-04 - val_loss: 0.4448 - val_acc: 6.6800e-04
Epoch 2/50
11976/11976 [==============================] - 2s 137us/step - loss: 0.4369 - acc: 9.1850e-04 - val_loss: 0.4448 - val_acc: 6.6800e-04
Epoch 3/50
11976/11976 [==============================] - 1s 124us/step - loss: 0.4369 - acc: 9.1850e-04 - val_loss: 0.4448 - val_acc: 6.6800e-04
Epoch 4/50
11976/11976 [==============================] - 2s 134us/step - loss: 0.4369 - acc: 9.1850e-04 - val_loss: 0.4448 - val_acc: 6.6800e-04
Epoch 5/50
11976/11976 [==============================] - 1s 124us/step - loss: 0.4369 - acc: 9.1850e-04 - val_loss: 0.4448 - val_acc: 6.6800e-04
Epoch 6/50
11976/11976 [==============================] - 1s 124us/step - loss: 0.4369 - acc: 9.1850e-04 - val_loss: 0.4448 - val_acc: 6.6800e-04
Epoch 7/50
11976/11976 [==============================] - 1s 124us/step - l

# Predict using LSTM Model 2

In [20]:
predict2 = model2.predict(x_test)

In [21]:
predict2

array([[0.       , 0.7615942, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7615942, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7615942, 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.7615942, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7615942, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.7615942, 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [24]:
eval2 = model2.evaluate(x_test, y_test, verbose=1)

2994/2994 [==============================] - 0s 33us/step


In [25]:
print('Test score:', eval2[0])
print('Test accuracy:', eval2[1])

Test score: 0.4448013137640281
Test accuracy: 0.0006680026720106881
